In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [3]:
def rho_shock_tube(x, rhoL, rhoR):
    rho = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    rho[mask1] = rhoL
    rho[mask2] = rhoR
    return rho

In [4]:
def u_shock_tube(x, uL, uR):
    u = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    u[mask1] = uL
    u[mask2] = uR
    return u

In [5]:
def P_shock_tube(x, PL, PR):
    P = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    P[mask1] = PL
    P[mask2] = PR
    return P

In [12]:
nx = 81
L = 10.0
dx = 0.25
dt = 0.0002
gamma = 1.4
rhoL = 1.0
rhoR = 0.125
uL = 0.0
uR = 0.0
PL = 100.0
PR = 10.0
nt = 0.01

x = numpy.linspace(-L, L, num=nx)
eT0 = P0 / rho0

rho0 = rho_shock_tube(x, rhoL, rhoR)
u0 = u_shock_tube(x, uL, uR)
P0 = P_shock_tube(x, PL, PR)

In [14]:
def flux2(rho, u, eT, gamma):
    u1 = rho
    u2 = rho * u
    u3 = rho * eT
    F = numpy.array([u2],
                    [u2**2 / u1 + (gamma - 1) * (u3 - 0.5 * u2**2 / u1)],
                    [(u3 + (gamma - 1) * (u3 - 0.5 * u2**2 / u1)) * u2 / u1])
    return F

In [15]:
def flux1(rho, u, eT, gamma):
    u1 = rho
    u2 = rho * u
    u3 = rho * eT
    U = numpy.array([u1],
                    [u2],
                    [u3])
    return U

In [ ]:
def maccormack(rho0, nt, dt, dx, bc_values, *args):
    u = u0.copy()
    u_rich = u.copy()
    for n in range(nt):
        F = flux1(u, *args)
        u_rich[1:-1] = (u[2:] -
                          dt / dx * (F[2:] - F[1:-1]))
        F = flux(rho_star, *args)
        rho[1:-1] = 0.5 * (rho[1:-1] + rho_star[1:-1] -
                           dt / dx * (F[1:-1] - F[:-2]))
        rho[0] = bc_values[0]
        rho[-1] = bc_values[1]
    return rho